In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append("..")

import os
import optuna
import torch
import wandb
import gc

from src.training import train_model
from src.models.model_unet import UNet
from src.dataprocess.torch_dataset_SNSD import DenoisingSpectrogramDataset
from torch.utils.data import DataLoader

# Path to Dataset
DATA_DIR = "/media/gabriel/500 BKP/datasets/SNSD-DataSet/preprocessed"
CHECKPOINTS_DIR = os.path.dirname(os.getcwd()) + "/checkpoints"


/home/gabriel/miniconda3/envs/venv_audio/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: gfiuzaa (gfiuzaa-universidade-federal-do-rio-de-janeiro). Use `wandb login --relogin` to force relogin


True

In [4]:
# Dataset
TARGET_FRAMES = 1024
train_dataset = DenoisingSpectrogramDataset(DATA_DIR, 'train', TARGET_FRAMES)
val_dataset = DenoisingSpectrogramDataset(DATA_DIR, 'validation', TARGET_FRAMES)

# Optuna
def objective(trial):
    # Search Space
    base_filters = trial.suggest_categorical('base_filters', [16, 32])
    depth = trial.suggest_int('depth', 3, 5)
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.3)
    lr = trial.suggest_float('lr', 1e-5, 1e-3, log=True)
    batch_size = trial.suggest_categorical('batch_size', [8, 16])

    # wandb logging
    wandb.init(project='audio-denoising', config={
        'base_filters': base_filters,
        'depth': depth,
        'dropout_rate': dropout_rate,
        'lr': lr,
        'batch_size': batch_size
    }, reinit=True)

    # Dataloader
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, persistent_workers=False)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, persistent_workers=False)

    # Model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = UNet(
        in_channels=1,
        out_channels=1,
        base_filters=base_filters,
        depth=depth,
        dropout_rate=dropout_rate
    ).to(device)

    # Optimizer and loss
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=2)
    criterion = torch.nn.MSELoss()

    # Train
    model, train_losses, val_losses, best_val_loss = train_model(
        model=model,
        optimizer=optimizer,
        criterion=criterion,
        scheduler=scheduler,
        device=device,
        train_loader=train_loader,
        train_dataset=train_dataset,
        val_loader=val_loader,
        val_dataset=val_dataset,
        num_epochs=100,
        early_stopping_patience=5,
        audio_preview=False,
        trial=trial,
        save_dir=CHECKPOINTS_DIR
    )

    # Always clean up
    if 'model' in locals():
        model.cpu()
        del model
    if 'optimizer' in locals():
        del optimizer
    if 'scheduler' in locals():
        del scheduler
    if 'train_loader' in locals():
        del train_loader
    if 'val_loader' in locals():
        del val_loader

    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    gc.collect()

    wandb.finish()

    return best_val_loss

In [5]:
study = optuna.create_study(
    study_name="audio-denoising",
    direction="minimize",
    storage="sqlite:///optuna_trials.db",
    load_if_exists=False,
)
study.optimize(objective, n_trials=50, timeout=8*60*60)  # 8 hours

[I 2025-04-27 13:52:09,955] A new study created in RDB with name: audio-denoising



🔁 Epoch 1/100


✅ Best model saved: /home/gabriel/VSProjects/audio_analysis/DenoisingAutoencoder/checkpoints/model_trial_0.pth (val loss: 0.0321)
📉 Train Loss: 0.0836 | Val Loss: 0.0321
🔧 Learning Rate: 0.000040

🔁 Epoch 2/100


✅ Best model saved: /home/gabriel/VSProjects/audio_analysis/DenoisingAutoencoder/checkpoints/model_trial_0.pth (val loss: 0.0290)
📉 Train Loss: 0.0421 | Val Loss: 0.0290
🔧 Learning Rate: 0.000040

🔁 Epoch 3/100


✅ Best model saved: /home/gabriel/VSProjects/audio_analysis/DenoisingAutoencoder/checkpoints/model_trial_0.pth (val loss: 0.0247)
📉 Train Loss: 0.0347 | Val Loss: 0.0247
🔧 Learning Rate: 0.000040

🔁 Epoch 4/100


...

✅ Best model saved: /home/gabriel/VSProjects/audio_analysis/DenoisingAutoencoder/checkpoints/model_trial_1.pth (val loss: 0.0675)
📉 Train Loss: 0.0759 | Val Loss: 0.0675
🔧 Learning Rate: 0.000011

🔁 Epoch 23/100


✅ Best model saved: /home/gabriel/VSProjects/audio_analysis/DenoisingAutoencoder/checkpoints/model_trial_1.pth (val loss: 0.0614)
📉 Train Loss: 0.0724 | Val Loss: 0.0614
🔧 Learning Rate: 0.000011

🔁 Epoch 24/100


✅ Best model saved: /home/gabriel/VSProjects/audio_analysis/DenoisingAutoencoder/checkpoints/model_trial_1.pth (val loss: 0.0591)
📉 Train Loss: 0.0694 | Val Loss: 0.0591
🔧 Learning Rate: 0.000011

🔁 Epoch 25/100


✅ Best model saved: /home/gabriel/VSProjects/audio_analysis/DenoisingAutoencoder/checkpoints/model_trial_1.pth (val loss: 0.0561)
📉 Train Loss: 0.0664 | Val Loss: 0.0561
🔧 Learning Rate: 0.000011

🔁 Epoch 26/100


Train Epoch:  54%|█████▍    | 83/154 [00:30<00:26,  2.70it/s]

In [ ]:
import optuna
import pandas as pd

def export_study_to_csv(study_path, study_name, out_csv="optuna_trials.csv"):
    study = optuna.load_study(study_name=study_name, storage=study_path)

    rows = []
    for trial in study.trials:
        row = {
            "trial": trial.number,
            "val_loss": trial.value,
            "model_path": trial.user_attrs.get("best_model_path", "N/A")
        }
        # Add hyperparameters
        row.update(trial.params)
        rows.append(row)

    df = pd.DataFrame(rows)
    df = df.sort_values("val_loss")
    df.to_csv(out_csv, index=False)

    print(f"✅ Exported {len(df)} trials to {out_csv}")

In [ ]:
export_study_to_csv(
    study_path="sqlite:///optuna_trials.db",
    study_name="audio-denoising",
    out_csv=os.path.dirname(os.getcwd()) + "/tuning/optuna_trials.csv"
)